In [1]:
import pandas as pd
import numpy as np
np.random.seed(8)
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
telemetry = pd.read_csv('/content/drive/MyDrive/aitomatic/PdM_telemetry.csv')
errors = pd.read_csv('/content/drive/MyDrive/aitomatic/PdM_errors.csv')
failures = pd.read_csv('/content/drive/MyDrive/aitomatic/PdM_failures.csv')
machines = pd.read_csv('/content/drive/MyDrive/aitomatic/PdM_machines.csv')
telemetry['datetime'] = pd.to_datetime(telemetry['datetime'], format="%Y-%m-%d %H:%M:%S")
errors['datetime'] = pd.to_datetime(errors['datetime'], format="%Y-%m-%d %H:%M:%S")
failures['datetime'] = pd.to_datetime(failures['datetime'], format="%Y-%m-%d %H:%M:%S")

In [3]:
error_count = pd.get_dummies(errors)
features = telemetry.merge(error_count, on=['datetime', 'machineID'], how='left')
features = features.merge(machines, on=['machineID'], how='left').fillna(0.0)
features = features.groupby(['machineID','datetime']).max().reset_index()
features = features.merge(failures, on=['datetime', 'machineID'], how='left').fillna('none')
label2int = {"none":0,"comp1":1,"comp2":1,"comp3":1,"comp4":1}
n_classes = len(label2int)
features["failure"] = features["failure"].apply(lambda x: label2int[x])
features

,machineID,datetime,volt,rotate,pressure,vibration,errorID_error1,errorID_error2,errorID_error3,errorID_error4,errorID_error5,model,age,failure
0,1,2015-01-01 06:00:00,176.217853,418.504078,113.077935,45.087686,0.0,0.0,0.0,0.0,0.0,model3,18,0
1,1,2015-01-01 07:00:00,162.879223,402.747490,95.460525,43.413973,0.0,0.0,0.0,0.0,0.0,model3,18,0
2,1,2015-01-01 08:00:00,170.989902,527.349825,75.237905,34.178847,0.0,0.0,0.0,0.0,0.0,model3,18,0
3,1,2015-01-01 09:00:00,162.462833,346.149335,109.248561,41.122144,0.0,0.0,0.0,0.0,0.0,model3,18,0
4,1,2015-01-01 10:00:00,157.610021,435.376873,111.886648,25.990511,0.0,0.0,0.0,0.0,0.0,model3,18,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876137,100,2016-01-01 02:00:00,179.438162,395.222827,102.290715,50.771941,0.0,0.0,0.0,0.0,0.0,model4,5,0
876138,100,2016-01-01 03:00:00,189.617555,446.207972,98.180607,35.123072,0.0,0.0,0.0,0.0,0.0,model4,5,0
876139,100,2016-01-01 04:00:00,192.483414,447.816524,94.132837,48.314561,0.0,0.0,0.0,0.0,0.0,model4,5,0
876140,100,2016-01-01 05:00:00,165.475310,413.771670,104.081073,44.835259,0.0,0.0,0.0,0.0,0.0,model4,5,0


In [4]:
milestone = '2015-11-15 01:00:00'
train = features[features["datetime"] < pd.to_datetime(milestone)]
test = features[features["datetime"] >= pd.to_datetime(milestone)]
train.describe()

,machineID,volt,rotate,pressure,vibration,errorID_error1,errorID_error2,errorID_error3,errorID_error4,errorID_error5,age,failure
count,762736.000000,762736.000000,762736.000000,762736.000000,762736.000000,762736.000000,762736.000000,762736.000000,762736.000000,762736.000000,762736.000000,762736.000000
mean,50.499608,170.785975,446.665698,100.836196,40.387587,0.001185,0.001122,0.000962,0.000819,0.000417,11.330081,0.000871
std,28.866222,15.504913,52.623687,11.026277,5.371539,0.034406,0.033482,0.031006,0.028614,0.020414,5.827596,0.029492
min,1.000000,97.333604,138.432075,51.237106,14.877054,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,25.000000,160.320304,412.368235,93.489419,36.779333,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,0.000000
50%,50.000000,170.610005,447.648021,100.409207,40.239140,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,0.000000
75%,75.000000,181.019007,482.200737,107.532908,43.792310,0.000000,0.000000,0.000000,0.000000,0.000000,16.000000,0.000000
max,100.000000,250.870453,695.020984,185.951998,76.791072,1.000000,1.000000,1.000000,1.000000,1.000000,20.000000,1.000000


In [5]:
features["failure"].value_counts()

0    875381
1       761
Name: failure, dtype: int64

In [7]:
train[['volt', 'rotate', 'pressure', 'vibration','failure']].groupby('failure').agg([np.mean,np.std,min,max])

volt                                         rotate             \
               mean        std         min         max        mean        std   
failure                                                                         
0        170.781573  15.501576   97.333604  250.870453  446.687720  52.608408   
1        175.838769  18.278858  115.638391  235.493717  421.390826  63.096921   

                                   pressure                                    \
                min         max        mean        std        min         max   
failure                                                                         
0        138.432075  695.020984  100.832060  11.021828  51.237106  185.951998   
1        247.946542  619.694725  105.583237  14.557577  74.239326  168.992513   

         vibration                                  
              mean       std        min        max  
failure                                             
0        40.385092  5.369532  14.877054  76.791072  
1        43.251541  6.742013  25.839522  63.246018

In [9]:
for field in ['machineID','age','model']:
  df_mean = train[['volt', 'rotate', 'pressure', 'vibration',field]].groupby(field).mean()
  df_mean = df_mean.add_prefix('mean_')
  df_std = train[['volt', 'rotate', 'pressure', 'vibration',field]].groupby(field).mean()
  df_std = df_std.add_prefix('std_')
  df = df_mean.join(df_std)
  rate = train.groupby(field).failure.agg([sum,len])
  df[f"{field}_rate"] = rate['sum']/ rate['len']
  df.to_csv(f"/content/drive/MyDrive/aitomatic/{field}_origin.csv")

In [10]:
train.to_csv("/content/drive/MyDrive/aitomatic/train_origin.csv",index=False)
test.to_csv("/content/drive/MyDrive/aitomatic/test_origin.csv",index=False)

In [11]:
df

,mean_volt,mean_rotate,mean_pressure,mean_vibration,std_volt,std_rotate,std_pressure,std_vibration,model_rate
model,,,,,,,,,
model1,170.752424,446.642272,101.239051,40.435168,170.752424,446.642272,101.239051,40.435168,0.001327
model2,170.781330,446.716595,101.186258,40.384975,170.781330,446.716595,101.186258,40.384975,0.001118
model3,170.823943,446.482849,100.653863,40.400357,170.823943,446.482849,100.653863,40.400357,0.000742
model4,170.763692,446.850363,100.648209,40.351217,170.763692,446.850363,100.648209,40.351217,0.000651
